# Recipe Recommender Pipeline Demonstration  
### YOLO → Ingredient Normalization → Recipe Matching

This notebook demonstrates how to use the **RecipeRecommender** pipeline, which fully integrates:

- **YOLOv8 ingredient detection**
- **Canonical ingredient normalization**
- **Normalized recipe database (pre-computed PKL)**
- **Simple and predictable recipe matching**
- **End-to-end pipeline for backend consumption**

This notebook is intended as a handoff for the teammate implementing:

- Backend API (FastAPI/Flask)
- UI (Streamlit, HTML/JS, Mobile, etc.)
- Docker deployment
- Basically, getting the app to have an UI and able to spin up & call the recipe recommender located below

## Below is a demonstration of RecipeRecommender (src/backend)

### Step 1: Import Recommender

In [1]:
from src.backend.recipe_recommender import RecipeRecommender

# Initialize the unified pipeline
rr = RecipeRecommender()

print("RecipeRecommender initialized with", len(rr.recipe_dict), "recipes.")


[RecipeRecommender] Loaded 13244 recipes.
RecipeRecommender initialized with 13244 recipes.


### Step 2: Run Single Image Recommendation

In [ ]:
from src.backend.recipe_recommender import RecipeRecommender
import os, random

rr = RecipeRecommender()

folder = "data/fridge_photos/test/images"
img = os.path.join(folder, random.choice(os.listdir(folder)))

result = rr.recommend(img, top_k=10)

# === Get the #1 recipe details in one call ===
details = rr.get_recipe_details(result, index=0)

print(details["title"])
print(details["ingredients_raw"])
print(details["instructions"])

[RecipeRecommender] Loaded 13244 recipes.

=== Pipeline Start ===
Image: data/fridge_photos/test/images/DSC_5959_JPG_jpg.rf.2063d00e55afc5401316f5d313b03bfe.jpg

image 1/1 /home/platinumfish/Desktop/6700/project/ml-app-deployment-proj/data/fridge_photos/test/images/DSC_5959_JPG_jpg.rf.2063d00e55afc5401316f5d313b03bfe.jpg: 640x640 1 bread, 1 butter, 1 eggs, 1 green_beans, 1 ground_beef, 1 heavy_cream, 1 mushrooms, 1 onion, 1 spinach, 1 strawberries, 2 sweet_potatos, 363.8ms
Speed: 7.2ms preprocess, 363.8ms inference, 5.8ms postprocess per image at shape (1, 3, 640, 640)

[1] Raw YOLO detections: ['butter', 'bread', 'mushrooms', 'spinach', 'eggs', 'green_beans', 'onion', 'heavy_cream', 'strawberries', 'sweet_potato', 'sweet_potato', 'ground_beef']
[2] Canonical fridge items: ['bread', 'butter', 'eggs', 'green_beans', 'ground_beef', 'heavy_cream', 'mushrooms', 'onion', 'spinach', 'strawberry', 'sweet_potato']

=== Top Matches ===
- Grilled Vanilla French Toast (score=3.00)
  ingredients: 

### Example run for top 5 recipes & all their info

In [10]:
from src.backend.recipe_recommender import RecipeRecommender
import os, random
from pprint import pprint

# Initialize recommender
rr = RecipeRecommender()

# Select a random fridge image
folder = "data/fridge_photos/test/images"
test_image = os.path.join(folder, random.choice(os.listdir(folder)))

# Run recommendation
result = rr.recommend(test_image, top_k=5)

# Extract full details for top 5 results
print("\n=== FULL DETAILS FOR TOP 5 RECIPES ===\n")

for i in range(5):
    details = rr.get_recipe_details(result, index=i)
    print(f"--- Recipe #{i+1} ---")
    print("TITLE:", details["title"])
    print("SCORE:", details["score"])
    
    print("\nCLEANED INGREDIENTS:")
    pprint(details["cleaned_ingredients"])

    print("\nNORMALIZED INGREDIENTS:")
    pprint(details["normalized_ingredients"])

    print("\nRAW INGREDIENT TEXT:")
    pprint(details["ingredients_raw"])

    print("\nINSTRUCTIONS:")
    print(details["instructions"])

    print("\n")


[RecipeRecommender] Loaded 13244 recipes.

=== Pipeline Start ===
Image: data/fridge_photos/test/images/DSC_6079_JPG_jpg.rf.88395cd136be2e1dc42e71618905d597.jpg

image 1/1 /home/platinumfish/Desktop/6700/project/ml-app-deployment-proj/data/fridge_photos/test/images/DSC_6079_JPG_jpg.rf.88395cd136be2e1dc42e71618905d597.jpg: 640x640 2 beefs, 1 blueberries, 1 cheese, 1 chicken_breast, 1 eggs, 1 flour, 1 green_beans, 1 lime, 1 onion, 1 potato, 1 spinach, 375.9ms
Speed: 7.0ms preprocess, 375.9ms inference, 3.7ms postprocess per image at shape (1, 3, 640, 640)

[1] Raw YOLO detections: ['onion', 'eggs', 'spinach', 'flour', 'green_beans', 'cheese', 'potato', 'lime', 'blueberries', 'chicken_breast']
[2] Canonical fridge items: ['blueberries', 'cheese', 'chicken_breast', 'eggs', 'flour', 'green_beans', 'lime', 'onion', 'potato', 'spinach']

=== Top Matches ===
- Spinach Pie (score=3.00)
  ingredients: ['eggs', 'onion', 'parmesan', 'spinach']
- Potato-Chip Frittatas (score=3.00)
  ingredients: ['

### View Results Clearly for 1 fridge item

In [12]:
from pprint import pprint

print("=== Canonical fridge items ===")
pprint(result["fridge_items"])

print("\n=== Top Recommendations ===")
for rec in result["recommendations"]:
    print(f"\n{rec['title']}  (score={rec['score']})")
    print("ingredients →", rec["normalized_ingredients"])


=== Canonical fridge items ===
['beef',
 'blueberries',
 'cheese',
 'eggs',
 'green_beans',
 'lime',
 'onion',
 'potato',
 'spinach']

=== Top Recommendations ===

Spinach Pie  (score=3.0)
ingredients → ['eggs', 'onion', 'parmesan', 'spinach']

Potato-Chip Frittatas  (score=3.0)
ingredients → ['carrot', 'eggs', 'leek', 'onion', 'potato', 'salt']

Instant Pot Bisibelabath  (score=2.0)
ingredients → ['black peppercorns', 'onion', 'spinach', 'whole cloves', 'yellow mustard seeds']

Sweet Potato and Pecan Waffles  (score=2.0)
ingredients → ['eggs', 'maple syrup', 'pure vanilla extract', 'rolled oats', 'spinach']

Shrimp Tacos with Pineapple  (score=2.0)
ingredients → ['avocado', 'corn tortillas', 'lime', 'spinach']

Spiced Dal with Fluffy Rice and Salted Yogurt  (score=2.0)
ingredients → ['garlic', 'lime', 'spinach']

Kale or Chard Pie  (score=2.0)
ingredients → ['all purpose flour', 'eggs', 'onion', 'salt']

Paella with Tomatoes and Eggs  (score=2.0)
ingredients → ['eggs', 'onion', 'salt'

### Show Recipe Ingredient Structure

In [6]:
# View one random recipe and its normalized ingredients (from PKL)
import random

rand_title = random.choice(list(rr.recipe_dict.keys()))
rand_ingredients = rr.recipe_dict[rand_title]

print("Title:", rand_title)
print("Normalized Ingredients:", rand_ingredients)


Title: Grilled Steak Panzanella Salad with Tomato Vinaigrette
Normalized Ingredients: {'normalized': ['flaky sea salt', 'garlic'], 'cleaned': ['3 tablespoons balsamic vinegar', '1/4 1/2 cup plus 3 tablespoons olive oil, divided, plus more for brushing', '2 1/4 teaspoons kosher salt, divided', '1 1/8 teaspoons freshly ground black pepper, divided', '3 garlic cloves, divided', '1 (11 1/2-pound) hanger steak, center membrane removed, cut into 4 pieces', '1 1/2 pounds ripe beefsteak or heirloom tomatoes, cut into bite-size pieces', '2 medium zucchini, cut in half lengthwise', '4 thick slices country-style bread', '1 tablespoon Dijon mustard', '1 tablespoon white wine vinegar', '1 bunch Tuscan kale, ribs and stems removed, thinly sliced (about 3 1/2 cups)', '1 1/2 cup basil leaves, torn if large', 'Flaky sea salt'], 'ingredients_raw': "['3 tablespoons balsamic vinegar', '1/4 1/2 cup plus 3 tablespoons olive oil, divided, plus more for brushing', '2 1/4 teaspoons kosher salt, divided', '1 1/

### Simple Overlap Matching Example

In [7]:
# Demonstrating the simple 1-1 scoring mechanism
fridge_items = ["milk", "eggs", "flour"]

example_recipe = ["milk", "eggs", "sugar", "butter"]

score = rr.matcher.match(fridge_items, example_recipe)

print("Fridge items:      ", fridge_items)
print("Recipe ingredients:", example_recipe)
print("Match score:", score)


Fridge items:       ['milk', 'eggs', 'flour']
Recipe ingredients: ['milk', 'eggs', 'sugar', 'butter']
Match score: 2


## **Final Team Handoff Instructions**


## ✅ What Allen Completed (ML Pipeline)
### 1. YOLOv8 Ingredient Detection  
- Fully integrated with ultralytics  
- Works on all test fridge images  
- Produces accurate ingredient labels

### 2. Canonical Ingredient Normalization  
- Converts YOLO labels → clean canonical tokens  
- Uses fuzzy matching + canonical_vocab.json

### 3. Recipe Normalization (60k+ ingredients cleaned)  
- All recipes have preprocessed ingredient lists  
- Cached into:
  - `data/normalized_recipes.pkl` (runtime)
  - `data/cached_normalized.csv` (debug)

### 4. Recipe Matching  
- Simple overlap scoring restored  
- Stable and predictable recommendations  
- Pipeline ready for backend integration

### 5. Unified Recommender Object  
- Complete end-to-end pipeline:
- YOLO → canonical → normalized recipes → matching → top results
- Lives in:  
  **src/backend/recipe_recommender.py**

---

# What Teammate Needs To Build

## 1. Build REST API Backend (FastAPI recommended)
Endpoints:
- `POST /detect` → returns detected ingredients
- `POST /recommend` → returns top recipes
- `GET /health` → healthcheck (required for Docker)

Backend must:
- Handle image uploads
- Save images temporarily
- Call RecipeRecommender
- Return JSON responses
- Validate inputs

---

## 2. Build Frontend / UI
Tools allowed:
- Streamlit
- React
- HTML/JS frontend
- Flask/Bootstrap
- FastAPI + Jinja templates

UI should:
- Upload an image
- Preview image + YOLO detections
- Show recommended recipes
- Show matching ingredients
- Show missing ingredients

Optional:
- Multi-image mode
- Ingredient “pantry list”
- Dietary filters (vegan, dairy-free, etc.)

---

## 3. Docker Deployment
Team must build Dockerfile including:
```python
FROM python:3.10
RUN pip install ultralytics fastapi uvicorn torch ...
COPY src /app/src
COPY data /app/data
CMD ["uvicorn", "backend.main:app", "--host", "0.0.0.0", "--port", "8000"]
```

Deliverables:
- Dockerfile
- docker-compose.yml
- README for running locally

---

## 4. Final Integration & Testing
✔ API endpoints tested with sample images  
✔ Frontend tested with full pipeline  
✔ Docker image builds locally  
✔ Optional: deploy to cloud (Render, AWS, Lightsail, etc.)

---

# Summary

This notebook + codebase provides a **complete ML backend**.

Your teammate now only needs to:

### **Build the WEB BACKEND + UI + Docker deploy**  
Everything else (model, recipe logic, detection pipeline) is fully ready.

---
